In [ ]:
!pip install rdflib

In [1]:
from rdflib import Graph, Namespace, RDF, RDFS, Literal, URIRef

g = Graph()

ANN = Namespace("https://schema.org/annotation/")
GAME = Namespace("https://schema.org/game/")
AUDIO = Namespace("https://schema.org/audio/")
TEXT = Namespace("https://schema.org/text/")
STATS = Namespace("https://schema.org/stats/")
EMO = Namespace("https://schema.org/emotion/")


g.bind("ann", ANN)
g.bind("game", GAME)
g.bind("audio", AUDIO)
g.bind("text", TEXT)
g.bind("stats", STATS)
g.bind("emo", EMO)


classes = [
    ANN.Annotation,
    GAME.PlayerChoice,
    AUDIO.AudioSegment,
    TEXT.ViewerComment,
    STATS.AltChoice,
    TEXT.DependencyRelation,
    TEXT.Entity,
    AUDIO.SpectralFeatures
]

for cls in classes:
    g.add((cls, RDF.type, RDFS.Class))


properties = {
    ANN.hasPlayerChoice: GAME.PlayerChoice,
    ANN.hasAudioSegment: AUDIO.AudioSegment,
    ANN.hasViewerComment: TEXT.ViewerComment,
    GAME.hasAltChoice: STATS.AltChoice,
    AUDIO.hasDependency: TEXT.DependencyRelation,
    AUDIO.hasEntity: TEXT.Entity,
    AUDIO.hasSpectralFeatures: AUDIO.SpectralFeatures,
    TEXT.hasDependency: TEXT.DependencyRelation,
    TEXT.hasEntity: TEXT.Entity,


    ANN.annotationId: "string",
    ANN.annotationVersion: "int",
    ANN.gameId: "string",
    ANN.eventId: "string",
    ANN.eventType: "string",
    ANN.contextDescription: "string",
    ANN.choiceDescription: "string",
    ANN.consequenceType: "string",
    ANN.annotatorId: "string",
    ANN.annotationTimestamp: "datetime",


    GAME.selectedChoiceDesc: "string",
    GAME.selectedChoiceRate: "float",
    GAME.choiceRank: "int",
    GAME.globalChoiceRate: "float",
    GAME.totalResponses: "int",
    GAME.dataSource: "url",


    AUDIO.segmentId: "string",
    AUDIO.startTime: "float",
    AUDIO.endTime: "float",
    AUDIO.speakerId: "string",
    AUDIO.transcriptText: "string",
    AUDIO.tokens: "string[]",
    AUDIO.sentiment: "string",
    AUDIO.audioEmotion: "string",
    AUDIO.reactionIntensity: "int",
    AUDIO.mfccFeatures: "float[]",


    AUDIO.spectralCentroid: "float",
    AUDIO.spectralBandwidth: "float",
    AUDIO.spectralRolloff: "float",
    AUDIO.zeroCrossingRate: "float",
    AUDIO.chromaFeatures: "float[]",
    AUDIO.melEnergy: "float[]",


    TEXT.commentId: "string",
    TEXT.commentText: "string",
    TEXT.commentTokens: "string[]",
    TEXT.commentSentiment: "string",
    TEXT.upvoteCount: "int",
    TEXT.commentTimestamp: "datetime",


    STATS.altChoiceDesc: "string",
    STATS.altChoiceRate: "float",


    TEXT.entityText: "string",
    TEXT.entityType: "string",
    TEXT.startToken: "int",
    TEXT.endToken: "int",
    TEXT.kbLink: "url"
}


for prop, data_type in properties.items():
    g.add((prop, RDF.type, RDF.Property))
    g.add((prop, RDFS.range, Literal(data_type) if isinstance(data_type, str) else data_type))

anno_uri = URIRef(ANN["anno1"])


g.add((anno_uri, RDF.type, ANN.Annotation))
g.add((anno_uri, ANN.annotationId, Literal("1.0")))
g.add((anno_uri, ANN.gameId, Literal("Mass Effect 3")))
g.add((anno_uri, ANN.eventType, Literal("moral_choice")))


player_choice_uri = URIRef(ANN["playerChoice1"])
g.add((anno_uri, ANN.hasPlayerChoice, player_choice_uri))
g.add((player_choice_uri, RDF.type, GAME.PlayerChoice))
g.add((player_choice_uri, GAME.selectedChoiceDesc,
       Literal("The krogan cure will be spread, Mordin Solus sacrifices himself")))
g.add((player_choice_uri, GAME.selectedChoiceRate, Literal(0.96)))


audio_segment_uri = URIRef(ANN["audioSegment1"])
g.add((anno_uri, ANN.hasAudioSegment, audio_segment_uri))
g.add((audio_segment_uri, RDF.type, AUDIO.AudioSegment))
g.add((audio_segment_uri, AUDIO.transcriptText, Literal("He is going to be fine")))
g.add((audio_segment_uri, AUDIO.audioEmotion, Literal("sadness")))
g.add((audio_segment_uri, AUDIO.reactionIntensity, Literal(3)))


spectral_uri = URIRef(AUDIO["spectral1"])
g.add((audio_segment_uri, AUDIO.hasSpectralFeatures, spectral_uri))
g.add((spectral_uri, RDF.type, AUDIO.SpectralFeatures))
g.add((spectral_uri, AUDIO.spectralCentroid, Literal(450.2)))
g.add((spectral_uri, AUDIO.zeroCrossingRate, Literal(0.08)))


comment_uri = URIRef(ANN["comment1"])
g.add((anno_uri, ANN.hasViewerComment, comment_uri))
g.add((comment_uri, RDF.type, TEXT.ViewerComment))
g.add((comment_uri, TEXT.commentText,
       Literal("Glad you got the chance to experience such a heart felt moment with Mordin")))


entity_uri = URIRef(TEXT["entity1"])
g.add((comment_uri, TEXT.hasEntity, entity_uri))
g.add((entity_uri, RDF.type, TEXT.Entity))
g.add((entity_uri, TEXT.entityText, Literal("Mordin")))
g.add((entity_uri, TEXT.entityType, Literal("character")))
g.add((entity_uri, TEXT.kbLink, Literal("https://dbpedia.org/page/Mordin_Solus")))


alt_choice_uri = URIRef(STATS["altChoice1"])
g.add((player_choice_uri, GAME.hasAltChoice, alt_choice_uri))
g.add((alt_choice_uri, RDF.type, STATS.AltChoice))
g.add((alt_choice_uri, STATS.altChoiceDesc,
       Literal("Sabotage the cure, save Mordin Solus")))
g.add((alt_choice_uri, STATS.altChoiceRate, Literal(0.4)))




<Graph identifier=Nea8f58a1f2af448b9fc98a0efdc13aba (<class 'rdflib.graph.Graph'>)>

In [2]:
query1 = """
SELECT ?anno ?eventType
WHERE {
    ?anno a ann:Annotation ;
        ann:gameId "Mass Effect 3" ;
        ann:eventType ?eventType .
}
"""


query2 = """
SELECT ?segment ?transcript
WHERE {
    ?anno ann:hasAudioSegment ?segment .
    ?segment audio:audioEmotion "sadness" ;
             audio:transcriptText ?transcript .
}
"""


query3 = """
SELECT ?comment ?entityText
WHERE {
    ?anno ann:hasViewerComment ?comment .
    ?comment text:hasEntity ?entity .
    ?entity text:entityType "character" ;
            text:entityText ?entityText .
}
"""


query4 = """
SELECT ?choice ?desc ?rate
WHERE {
    ?anno ann:hasPlayerChoice ?choice .
    ?choice game:selectedChoiceRate ?rate ;
            game:selectedChoiceDesc ?desc .
    FILTER(?rate > 0.5)
}
"""




print("Аннотации для Mass Effect 3:")
for row in g.query(query1, initNs={"ann": ANN}):
    print(f"- {row.anno}: {row.eventType}")

print("\nСегменты с эмоцией 'sadness':")
for row in g.query(query2, initNs={"ann": ANN, "audio": AUDIO}):
    print(f"- {row.transcript}")

print("\nУпоминания персонажей в комментариях:")
for row in g.query(query3, initNs={"ann": ANN, "text": TEXT}):
    print(f"- {row.entityText}")

print("\nПопулярные выборы (>50%):")
for row in g.query(query4, initNs={"ann": ANN, "game": GAME}):
    print(f"- {row.desc} ({row.rate})")



g.serialize("annotation_graph.ttl", format="turtle")

Аннотации для Mass Effect 3:
- https://schema.org/annotation/anno1: moral_choice

Сегменты с эмоцией 'sadness':
- He is going to be fine

Упоминания персонажей в комментариях:
- Mordin

Популярные выборы (>50%):
- The krogan cure will be spread, Mordin Solus sacrifices himself (0.96)


<Graph identifier=Nea8f58a1f2af448b9fc98a0efdc13aba (<class 'rdflib.graph.Graph'>)>

In [3]:
import pydot
from rdflib.tools.rdf2dot import rdf2dot

with open("annotation_graph.dot", "w") as f:
    rdf2dot(g, f)

(graph,) = pydot.graph_from_dot_file("annotation_graph.dot")
graph.write_png("annotation_graph.png")

FileNotFoundError: [WinError 2] "dot" not found in path.